In [1]:
!pip install pandas sastrawi nltk


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import re
import io
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords

# ==============================================================================
# 1. PERSIAPAN: Fungsi dan Library
# ==============================================================================

# Inisialisasi stemmer dari Sastrawi (hanya perlu dibuat sekali)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Ambil daftar stopwords Bahasa Indonesia dari NLTK
# Anda mungkin perlu menjalankan: import nltk; nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    words = text.split()
    cleaned_words = [word for word in words if word not in list_stopwords]
    return " ".join(cleaned_words)

# Fungsi untuk melakukan stemming
def stem_text(text):
    return stemmer.stem(text)

# ==============================================================================
# 2. SIMULASI DATA ANDA
# (Di sini saya menyalin data Anda ke dalam string. 
#  Di program Anda yang sebenarnya, Anda akan menggunakan pd.read_csv)
# ==============================================================================

data_string = """id_berita,kategori,judul,isi_berita,link
153055,Jatim,Ada Santri yang Diduga Dihukum untuk Cor Bangunan Musala Ambruk Ponpes Al Khoziny,"SIDOARJO,BANGSAONLINE.com- Proses pencarian korban para santri tertimbun reruntuhan musala ambruk di Pondok Pesantren (Ponpes) Al-Khoziny, Buduran, Sidoarjo, masih terus berlangsung hingga, Rabu (1/10/2025)siang. Seorang santri yang enggan disebut namanya mengaku jika santri lain yang melakukan pengecoran bangunan adalah sebuah hukuman bagi santri yang tidak mengikuti kegiatan. “Ambruknya pas salat berjemaah. Imamnya selamat, tapi banyak teman-teman saya yang tertimpa,” tutur santri tersebut, Rabu (1/10/2025).  Hingga kini, Tim SAR gabungan masih berjibaku menyisir puing-puing bangunan tiga lantai tersebut. Dari total ratusan santri, 120 sudah berhasil dievakuasi, namun 66 lainnya masih dalam pencarian. Di balik musibah memilukan ini, terselip kisah tentang keseharian santri di pondok. Salah satunya adalah kebiasaan pemberian hukuman bagi mereka yang tidak mengikuti kegiatan pesantren. Hukuman itu berupa ikut membantu pengecoran bangunan yang sedang dikerjakan. “Kalau ada santri yang ketahuan bolos, biasanya dihukum bantu ngecor. Tapi sebenarnya kami hanya ikut saja, karena tukangnya banyak. Santri tidak wajib, cuma disuruh bantu kalau kena hukuman,” jelas santri yang sudah enam tahun mondok tersebut.  Beruntung, saat musala itu ambruk, ia sedang tidak berada di lokasi. Ia sedang berada di luar pondok saat itu. ""Saya baru sampai pondok, musalanya sudah ambruk. Ratusan santri waktu itu sedang salat asar. Banyak yang tertimpa reruntuhan,” ujarnya. Cerita lain diungkap dari Abdul, keluarga salah satu korban. Keponakannya yang berasal dari Madura disebut sedang berada di dekat santri yang tengah membantu pengecoran sebelum musala itu runtuh. “Jadi lagi ngecor, jatuh, terus luka di wajah, giginya copot,” pungkasnya. (cat)",https://bangsaonline.com/berita/153055/ada-santri-yang-diduga-dihukum-untuk-cor-bangunan-musala-ambruk-ponpes-al-khoziny
153054,Jatim,"Warga Dringu Lapor Dugaan Penipuan Kavling, Polres Probolinggo Kota Siap Tindaklanjuti","PROBOLINGGO, BANGSAONLINE.com- Polres Probolinggo Kota menerima laporan dari warga Desa/Kecamatan Dringu, terkait dugaan penipuan oleh seorang pengembang tanah kavling. Hal tersebut disampaikan oleh Kasatreskrim Polres Probolinggo Kota, Iptu Zainal Arifin, Rabu (1/9/2025). ""Ada tiga orang yang melaporkan,"" ujarnya saat dikonfirmasi. Namun, ia menjelaskan bahwa pelapor belum membawa dokumen pendukung yang lengkap, sehingga mereka disarankan untuk kembali dengan berkas yang diperlukan.  ""Mereka tidak membawa dokumen saat laporan, sehingga kita sarankan untuk kembali lagi,"" katanya. Meski laporan masih dalam tahap pengaduan, pihak kepolisian memastikan akan tetap melakukan penyelidikan. ""Nanti tetap kita lidik,"" ucap Zainal.  Sebelumnya, puluhan warga Desa Dringu mendatangi Mapolres Probolinggo Kota untuk menyampaikan keluhan. Mereka mengaku telah membeli tanah kavling dari seorang pengembang, namun hingga kini belum menerima sertifikat kepemilikan. ""Puluhan warga membeli tanah kavling. Namun sampai saat ini mereka tidak mendapatkan sertifikat,"" kata salah satu warga, Kholil. Warga berharap agar kasus ini segera ditindaklanjuti secara serius oleh pihak kepolisian, mengingat banyak warga yang merasa dirugikan. ""Warga minta agar polres menindaklanjuti kasus ini sampai selesai,"" pungkasnya. (ugi/mar)",https://bangsaonline.com/berita/153054/warga-dringu-lapor-dugaan-penipuan-kavling-polres-probolinggo-kota-siap-tindaklanjuti
153053,Jatim,"Hadiri Pengukuhan Pengurus Kormi Kota Kediri, Gus Qowim Dorong Olahraga untuk Semua Usia","KOTA KEDIRI, BANGSAONLINE.com- Wakil Wali Kota Kediri, Qowimuddin Thoha atau yang akrab disapa Gus Qowim, menghadiri pengukuhan pengurus Komite Olahraga Masyarakat Indonesia (Kormi) periode 2025-2029, Rabu (1/10/2025). Pengukuhan dilakukan oleh Plt Ketua Kormi Jatim, Andun Sudjiatmoko. Abdian Asgi Sukmana kembali dipercaya sebagai Ketua Kormi Kota Kediri untuk periode kedua. ""Selamat dan sukses atas dikukuhkan kembali Ketua Kormi Kota Kediri periode 2025-2029 Bapak Abdian Asgi Sukmana beserta jajaran pengurus. Saya juga sampaikan terima kasih atas berbagai dedikasi dan perjuangan segenap pengurus KORMI yang telah membawa organisasi ini meraih banyak prestasi,"" kata Gus Qowim.  Ia menambahkan, meski baru aktif selama 6 tahun, Kormi Kota Kediri telah berpartisipasi dalam berbagai kompetisi tingkat nasional dan internasional. Salah satu prestasi yang dibanggakan adalah keikutsertaan dalam ajang Tafisa Internasional dengan membawa olahraga tradisional khas Kediri, Takraw Keronjang. Saat ini, Kormi Kota Kediri menaungi 24 induk organisasi olahraga, mulai dari pushbike, Komunitas Sepeda Tua Indonesia, hingga senam tera.  Menurut Gus Qowim, keberagaman ini menunjukkan bahwa olahraga relevan untuk semua usia dan menjadi wadah penting bagi para atlet untuk terus berprestasi. ""Mari kita jadikan momentum ini sebagai penyemangat untuk memasyarakatkan olahraga dan mengolahragakan masyarakat. Serta membangun Kota Kediri yang MAPAN,"" tuturnya. Usai pengukuhan, dilakukan penyerahan pataka dari Plt Ketua Kormi Jatim kepada Ketua Kormi Kota Kediri, dilanjutkan dengan penyematan jaket oleh Wakil Wali Kota Kediri sebagai simbol dukungan dan semangat baru bagi kepengurusan yang baru dikukuhkan. (uji/mar)",https://bangsaonline.com/berita/153053/hadiri-pengukuhan-pengurus-kormi-kota-kediri-gus-qowim-dorong-olahraga-untuk-semua-usia
"""

# Membaca string seolah-olah file CSV
# GANTI BAGIAN INI dengan pd.read_csv('nama_file_anda.csv') untuk data asli Anda
df = pd.read_csv(io.StringIO(data_string))

print("Data Awal Dimuat:")
print(df.head())
print("-" * 50)

# ==============================================================================
# 3. PROSES PENGOLAHAN TEKS (NLP PIPELINE)
# ==============================================================================

# a. Case Folding (mengubah ke huruf kecil)
print("Memulai Case Folding...")
df['isi_clean'] = df['isi_berita'].str.lower()

# b. Menghilangkan simbol, angka, dan tanda baca
print("Memulai Penghapusan Simbol...")
df['isi_nosymbol'] = df['isi_clean'].str.replace(r'[^a-z\s]', '', regex=True)

# c. Menghilangkan stopwords
print("Memulai Stopword Removal...")
df['isi_no_stopwords'] = df['isi_nosymbol'].apply(remove_stopwords)

# d. Stemming (ini mungkin memakan waktu paling lama)
print("Memulai Stemming... (mohon tunggu)")
df['isi_stemmed'] = df['isi_no_stopwords'].apply(stem_text)

# e. Tokenisasi (memecah menjadi daftar kata)
print("Memulai Tokenisasi...")
df['isi_tokens'] = df['isi_stemmed'].apply(lambda x: x.split())

print("Semua proses selesai!")
print("-" * 50)

# ==============================================================================
# 4. MEMBUAT DATAFRAME FINAL DAN MENYIMPAN
# ==============================================================================

# Pilih kolom yang ingin disimpan ke file CSV baru
kolom_final = [
    'id_berita', 'kategori', 'judul', 
    'isi_berita', 'link', 'isi_clean', 
    'isi_nosymbol', 'isi_stemmed', 'isi_tokens'
]

df_final = df[kolom_final]

# Simpan ke file 'hasil_tokenized.csv'
df_final.to_csv('hasil_tokenized_berita.csv', index=False)

print("File 'hasil_tokenized.csv' telah berhasil dibuat.")
print("\nContoh 5 baris pertama data hasil proses:")
print(df_final.head())

Data Awal Dimuat:
   id_berita kategori                                              judul  \
0     153055    Jatim  Ada Santri yang Diduga Dihukum untuk Cor Bangu...   
1     153054    Jatim  Warga Dringu Lapor Dugaan Penipuan Kavling, Po...   
2     153053    Jatim  Hadiri Pengukuhan Pengurus Kormi Kota Kediri, ...   

                                          isi_berita  \
0  SIDOARJO,BANGSAONLINE.com- Proses pencarian ko...   
1  PROBOLINGGO, BANGSAONLINE.com- Polres Probolin...   
2  KOTA KEDIRI, BANGSAONLINE.com- Wakil Wali Kota...   

                                                link  
0  https://bangsaonline.com/berita/153055/ada-san...  
1  https://bangsaonline.com/berita/153054/warga-d...  
2  https://bangsaonline.com/berita/153053/hadiri-...  
--------------------------------------------------
Memulai Case Folding...
Memulai Penghapusan Simbol...
Memulai Stopword Removal...
Memulai Stemming... (mohon tunggu)
Memulai Tokenisasi...
Semua proses selesai!
--------------------